In [11]:
!sudo apt install tesseract-ocr
!pip install pytesseract --quiet
!pip install opencv-python --quiet
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok
!ngrok authtoken 1wxMCTAfnBPTRRbFr8MpAreDsKO_5oPd8JRMAPqAfBUeNkruq

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [12]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml

--2022-11-19 08:39:04--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75482 (74K) [text/plain]
Saving to: ‘haarcascade_russian_plate_number.xml.2’

haarcascade_russian 100%[===================>]  73.71K  --.-KB/s    in 0.005s  

2022-11-19 08:39:04 (13.8 MB/s) - ‘haarcascade_russian_plate_number.xml.2’ saved [75482/75482]



In [13]:
%%writefile app.py
#import prebuilt Packages
import numpy as np
import cv2
import pytesseract
import streamlit as st
from PIL import Image
st.title("CAR NUMBER PLATE DETECTION AND RECOGNITION")
#taking image input
img = st.sidebar.file_uploader("Choose an image")
custom_config = r'--psm 10--oem 3'
if img is not None:
    img = Image.open(img)
    st.image(img,caption = "Uploaded Image")
    if st.button("PREDICT"):
        img = np.array(img)
        gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        gray_img = cv2.bilateralFilter(gray_img,15,90,90)
        NUM_PLATE_MODEL = cv2.CascadeClassifier("/content/haarcascade_russian_plate_number.xml")
        NUM_PLATE_DEMO = NUM_PLATE_MODEL.detectMultiScale(img,1.1,10)
        flag = 0 
        for (x,y,w,h) in NUM_PLATE_DEMO:
            flag = 1
            cv2.rectangle(img,(x,y),(x+w,y+h),[255,0,0],4)
            st.title("NUMBER PLATE DETECTED ")
            st.image(img)
            roi = gray_img[y:y+h,x:x+w]
            img = img[y:y+h,x:x+w]
            # Finding text from image using OCR
            text = pytesseract.image_to_string(roi,config = custom_config)
            st.title(" EXTRACTED NUMBER PLATE ")
            st.image(img)
            st.title(f" Extracted Number : {text}")
            # For finding each individual Chracter 
            Boxes = pytesseract.image_to_boxes(roi)
            hImg,wImg,d = img.shape
            for b in Boxes.splitlines():
                b = b.split()
                x,y,w,h = int(b[1]),int(b[2]),int(b[3]),int(b[4])
                cv2.rectangle(img,(x,hImg-y),(w,hImg-h),(0,0,255),2)
            st.title(" EXTRACTED NUMBER PLATE with Characters  ")
            st.image(img)
        else :
            if flag == 0:
                st.title(" NO Number Plate Detected")

    



Overwriting app.py


In [14]:
!streamlit run app.py&>/dev/null&
url = ngrok.connect(port='8501')
print(url)

PyngrokNgrokHTTPError: ignored